In [ ]:
#using Pkg
#Pkg.add("XLSX")
#Pkg.add("Distributions")
#Pkg.add("Lasso")
#Pkg.add("GLMNet")
#Pkg.add("MLJ")
#Pkg.add("MLBase")
#Pkg.add("HDMjl")
#import Pkg; Pkg.add("MLJScikitLearnInterface")
#Pkg.add("HypothesisTests")


In [1]:
using XLSX
using Distributions
using DataFrames
using Dates
using Plots
using Random
using LinearAlgebra
using LaTeXStrings
using Lasso
using MLBase
using Statistics
using GLMNet
using MLJ
using HDMjl
using GLM
using CSV


2 Lasso (8 points)

In [ ]:
df = DataFrame(XLSX.readtable("Districtwise_literacy_rates.xlsx", "2015_16_Districtwise"))
dropmissing!(df)

In [ ]:
Graph1=histogram(
    [df.MALE_LIT, df.FEMALE_LIT],       
    bins = 0:10:150,
    xlabel = "Literacy rate",
    ylabel = "Frequency",
    title = "Female and Male literacy rate",
    label = ["Male" "Female"],         
    ylim = (0, 625),
    alpha = 0.6
) 


savefig(Graph1, "Histogram.pdf")                        
#Como se ve en el gráfico, en India la distribución de la tasa de alfabetismo en hombres está sesgada a la izquierda, mientras que la de las mujeres parece que tiene una distribución normal. 
#Es decir, en el caso de los hombres hay más observaciones en el lado derecho de la distribución o es más probable que tengan tasas de alfabetismo de más de 70% y unos pocos casos en los que el alfabetismo llega al rededor de 50% y 60%.
#En el caso de las mujeres, parece que la probabilidad de tener una alta tasa de alfabetización es similar a la de tener una baja.
#Asimismo, hay una mayor cantidad de hombres que saben leer y escribir a comparacion de las mujeres.

Low-dimensional specification

In [4]:
Random.seed!(12)
y = df[:, "FEMALE_LIT"]
Z = select(df, Not([:FEMALE_LIT]))
column_names = names(Z)
y=Float64.(y)
df.GROWTHRATE = Float64.(df.GROWTHRATE)
df.SCH2 = Int.(df.SCH2)
df.SELE2 = Int.(df.SELE2)
df.CLS2 = Int.(df.CLS2)
df.TCH2 = Int.(df.TCH2)
df.SCOMP2 = Int.(df.SCOMP2)
df.SEXRATIO = Int.(df.SEXRATIO)
df.P_URB_POP = Float64.(df.P_URB_POP)
df.TOTPOPULAT = Float64.(df.TOTPOPULAT)



df.GROWTHRATE2=df.GROWTHRATE.^ 2
df.P_URB_POP2=df.P_URB_POP.^ 2
df.SEXRATIO2=df.SEXRATIO.^ 2
df.TOTPOPULAT2=df.TOTPOPULAT.^ 2
df.SCH2_cuadrado=df.SCH2.^ 2
df.SELE2_cuadrado=df.SELE2.^ 2
df.SCOMP2_cuadrado=df.SCOMP2.^ 2
df.CLS2_cuadrado=df.CLS2.^ 2
df.TCH2_cuadrado=df.TCH2.^ 2

basic_formula = @formula(FEMALE_LIT ~ 1 +STATNAME+GROWTHRATE+P_URB_POP+ SEXRATIO+TOTPOPULAT+SCH2+SELE2+SCOMP2+CLS2+TCH2)  
Z_base = modelmatrix(basic_formula, df);

train_sample = rand(Float64, size(df)[1]) .< 0.75
test_sample = .!(train_sample)
y_train, y_test = y[train_sample], y[test_sample]
X_train, X_test = Z_base[train_sample, :], Z_base[test_sample, :];
y_train=Float64.(y_train)
y_test=Float64.(y_test)

lr_base = lm(X_train, y_train);

basic_mse_testing = mean((GLM.predict(lr_base, X_test) - y_test) .^ 2)

basic_r2_testing = 1 - basic_mse_testing / var(y_test)
#Según el R cuadrado, parece que el modelo explica bien la variabilidad en la tasa de alfabetismo de las mujeres en India

0.48596087340293526

High-dimensional (flexible) specification

In [5]:

flex_formula = @formula(FEMALE_LIT ~ 1 +STATNAME+ GROWTHRATE + P_URB_POP + SEXRATIO + TOTPOPULAT + SCH2 + SELE2 + SCOMP2 + CLS2 + TCH2+GROWTHRATE2 + P_URB_POP2 + SEXRATIO2 + TOTPOPULAT2 + SCH2_cuadrado + SELE2_cuadrado + SCOMP2_cuadrado + CLS2_cuadrado + TCH2_cuadrado+(GROWTHRATE+GROWTHRATE2)*( P_URB_POP + SEXRATIO + TOTPOPULAT + SCH2 + SELE2 + SCOMP2 + CLS2 + TCH2))
Z_flex = modelmatrix(flex_formula, df);

X_train, X_test = Z_flex[train_sample, :], Z_flex[test_sample, :];

lr_flex = lm(X_train, y_train);

flexible_mse_testing = mean((GLM.predict(lr_flex, X_test) - y_test) .^ 2)
flexible_r2_testing = 1 - flexible_mse_testing / var(y_test)

flexible_r2_testing
#El R-cuadrado de estimar con OLS los terminos cuadraticos e interacciones es extremadamente negativo. El estimador OLS no es muy útil para estos casos con una gran cantidad de regresores.

-12.316659865854264

In [6]:
hdm_lasso = rlasso(Z_flex, y, intercept = false)

hdm_predictions = Z_flex * hdm_lasso["coefficients"];
R2_Lasso=1 - mean((hdm_predictions - y) .^ 2) / var(y)
R2_table = DataFrame(
    Modelo = ["Basic_OLS", "Flexible_OLS", "Lasso"],
    R² = [basic_r2_testing, flexible_r2_testing, R2_Lasso]
)
CSV.write("R2_table.csv", R2_table)
R2_Lasso
#Al estimar con Lasso el R cuadrado es incluso más alto que haciendo OLS al modelo base. Esto muestra que para casos de alta dimensionalidad Lasso es más util que hacer un OLS.

0.5576930233426556

In [ ]:
LassoRegressor = @load LassoRegressor pkg=MLJScikitLearnInterface
lambdas = [10000, 5000, 1000, 500, 100, 50, 10, 5, 1, 0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
num_coefs_nonzero = Int[]

for lambda in lambdas
    lasso_model = LassoRegressor(alpha=lambda, fit_intercept=false)
    lasso_machine = machine(lasso_model, X_train, y_train)
    fit!(lasso_machine, verbosity=0) 
    coefficients = fitted_params(lasso_machine).coef
    n_nonzero = sum(abs.(coefficients) .> 1e-10)
    push!(num_coefs_nonzero, n_nonzero)
  
end

results_df = DataFrame(
    lambda = lambdas,
    n_nonzero_coefs = num_coefs_nonzero,
)

p1= plot(lambdas, num_coefs_nonzero, 
          marker=:circle, linewidth=2,
          xlabel="λ", ylabel="Nonzero coefficients",
          title="Coefficients vs Lambda", legend=false)

CSV.write("Number of Nonzero coefficients.csv", results_df)         
savefig(p1, "Nonzero coefficients vs Lambda.pdf") 
#En el gráfico se ve que cuanto mayor sea el Lambda hay una menor cantidad de coeficientes diferente a cero. Es decir, se castiga mucho más y el modelo es más parsimonioso.

3. Potential Outcomes and RCTs (9 points)

3.1 Data Simulation (3 points)

In [8]:
using Random
using DataFrames
using Statistics
using Distributions
using HypothesisTests: EqualVarianceTTest, pvalue
using GLM
using GLMNet
using CSV

In [ ]:
Random.seed!(12345678)
n=1000
e=rand(Normal(0,1),n)
x1 = rand(Normal(0,1), n)
x2 = rand(Normal(5,2), n)
x3 = rand(Uniform(0,10), n)
x4 = rand(n)
D = Int.(rand(Bernoulli(0.5), n))
Y = 2 .* D .+ 0.5 .* x1 .- 0.3 .* x2 .+ 0.2 .* x3 .+ e

df = DataFrame(
    Y = Y,
    D = D,
    x1 = x1,
    x2 = x2,
    x3 = x3,
    x4 = x4,
    e = e
)
CSV.write("Data_simulation.csv", df)


In [ ]:
covariates = [:x1, :x2, :x3, :x4]

balance_table = DataFrame(
    Variable = String[],
    Mean_Control = Float64[],
    Mean_Treated = Float64[],
    Difference = Float64[],
    P_value = Float64[]
)

for x in covariates
    treated = df[df.D .== 1, x]
    control = df[df.D .== 0, x]

    test = EqualVarianceTTest(treated, control)

    mean_treated = mean(treated)
    mean_control = mean(control)
    diff = mean_treated - mean_control
    p_val = pvalue(test)

    push!(balance_table, (string(x), mean_control, mean_treated, diff, p_val))
end

display(balance_table)
#en la tabla de balance se ve que las diferencias son muy pequeñas y no significativas. Esto indica que al haberse asignado el tratamiento de forma aleatoria, las caracteristicas de las observaciones de ambos grupos son en promedio similares.
CSV.write("balance_table.csv", balance_table)

3.2 Estimating the Average Treatment Effect (3 points)


In [ ]:
model_1 = lm(@formula(Y ~ D), df)
model_2 = lm(@formula(Y ~ D+x1+x2+x3+x4), df)
#el coeficiente de la variable de tratamiento se mantiene con valores similares y significativos, pero controlando por las otras variables el coeficiente estimado se aleja un poco más del parámetro. El error estandar disminuye cuando se incluyen las otras variables.
CSV.write("model_1.csv", DataFrame(StatsBase.coeftable(model_1)))
CSV.write("model_2.csv", DataFrame(StatsBase.coeftable(model_2)))

3.3 LASSO and Variable Selection (3 points)


In [ ]:
y = df.Y                    
X = Matrix(select(df, [:x1, :x2, :x3, :x4]))  
fit = glmnetcv(X, y, alpha=1,intercept=true)
betas_l1 = fit.path.betas[:,argmin(fit.meanloss)];
Betas_df = DataFrame([(Symbol("Beta$(i)") => [betas_l1[i]]) for i in 1:length(betas_l1)]...)
CSV.write("Betas_lasso.csv", Betas_df)

In [ ]:
model_lasso = lm(@formula(Y ~ D+x1+x2+x3), df)
#Se escogen los 3 primero regresores y el coeficiente estimado se acerca más que en el caso con 4 controles, pero la diferencia no es tanta.
#Al ser un proceso generador de datos con tan pocas variables Lasso solo elimina una variable y es justo la que no esta en el PGD original. 
#La ventaja de Lasso podria ser que dismiuye el error estandar en este caso, lo que permite estimar con algo más de precision el parametro.
CSV.write("model_lasso.csv", DataFrame(StatsBase.coeftable(model_lasso)))